# Data EDA

* 데이터 라벨

|0|1|2|3|4|5|6|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|생활문화|스포츠|사회|세계|IT과학|경제|정치|

# Library import 단계

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Noise vs Non - Noise 분류
* Noise가 포함된 데이터가 1600개
  * 이때 Noise는 기사의 제목의 일부는 랜던함 `ASCII`로 변환함.
* Non - Noise가 포함된 데이터가 1200개
  * Non - Noise에서 제대로 Label된 데이터가 200개, label의 섞인 데이터가 1000개

In [3]:
# train_data 불러오기
data = pd.read_csv('./data/train.csv')
data.head()

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6


In [4]:
## 데이터 분포 확인.
label_distribution = []
for num in range(7):
    cond = data['target'] == num
    result = len(data[cond])
    label_distribution.append(result)
label_distribution


[397, 410, 388, 385, 406, 419, 395]

In [5]:
def mapping_label(df):
    # 딕셔너리로 매핑 정의
    label_map = {
        0:'생활문화',1:'스포츠',2:'사회',3:'세계',4:'IT과학',5:'경제',6:'정치'
    }
    
    # map 함수를 사용하여 변환
    df['target'] = df['target'].map(label_map)
    
    return df

# 사용
df = mapping_label(data)
df.head(3)

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,IT과학
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,세계
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",사회


In [14]:
## 현재 문장에서 ascii의 비율과, ascii의 비율 중 영어와 대문자가 있는 경우에 대한 확인.
def calculate_ascii(text):
    # 공백을 제외한 ASCII 문자들
    ascii_chars = [char for char in text if ord(char) < 128 and not char.isspace()]
    ascii_count = len(ascii_chars)
    ascii_ratio = ascii_count / len(text)

    # ASCII 문자들이 영어에 대한 
    english_chars = [char for char in ascii_chars if char.isalpha()]
    has_english = len(english_chars) > 0
    # 영어 관련 체크 영어 관련 내용이 있을경우
    is_english_only = all(char.isalpha() for char in ascii_chars) if has_english else False
    # 대문자 관련 계산 (영어가 하나라도 있을 때만 체크)
    is_all_uppercase = all(char.isupper() for char in english_chars) if has_english else False
    uppercase_count = sum(1 for char in ascii_chars if char.isupper())
    uppercase_ratio = uppercase_count / len(ascii_chars) if ascii_chars else 0
    
    return ascii_count, is_english_only, is_all_uppercase, ascii_ratio, uppercase_ratio

df['text'] = df['text'].str.replace('...', ' ').str.replace('…', ' ')
df['length'] = df['text'].map(len)
df['ascii_count'], df['is_english_ascii'], df['is_all_uppercase'], df['ascii_ratio'], df['uppercase_ratio'] = zip(*df['text'].apply(calculate_ascii))

# 결과 확인
print("ASCII 분석 결과:")
df[['text', 'ascii_count', 'is_english_ascii', 'is_all_uppercase', 'ascii_ratio', 'uppercase_ratio']].head()

ASCII 분석 결과:


,text,ascii_count,is_english_ascii,is_all_uppercase,ascii_ratio,uppercase_ratio
0,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,13,False,False,0.406250,0.307692
1,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,16,False,False,0.592593,0.375000
2,"m 김정) 자주통일 새,?r열1나가야1보",7,False,False,0.318182,0.000000
3,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,3,False,False,0.103448,0.000000
4,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,9,False,False,0.321429,0.333333


## 조건 별 결과 확인
* Text의 ASCII 비율이 20% 초과인 경우 : 1643
* ASCII 비율중 대문자 영어의 90% 이상인 경우 : 140
  * `미세먼지 싫어N` 이거 말고는 전부 정상적인 text(확인 완료)
* 전체 데이터셋에서 ASCII가 영어로만 구성되어 있는 샘플
  * `9/27`개 가 잘못되어 잇음
  * 해당 과정 중에 대문자의 비율이 0.8 이상은 `10개`
    * `IoT로 쓰레기통 관리해요`는 정상 문자 -> not 아스키
    * `L자Mk 삼성생D OK저축은P에 완승 연패 탈출`이 부분에 대해 관련해서는 잘못된 부분인데 비율이 0.857정도로 오히려 구분 안됨


In [41]:
cond1 = df['ascii_ratio'] >= 0.2
cond2 = df['uppercase_ratio'] < 0.8
cond3 = df['is_english_ascii']

In [16]:
high_ascii = df[~cond2]
len(high_ascii)


140

In [ ]:
# 두번째 확인 결련 코드
total_rows = len(high_ascii)
chunk_size = 20

for i in range(0, total_rows, chunk_size):
    chunk = high_ascii.iloc[i:i + chunk_size]
    print(f"\n=== {i+1}~{min(i+chunk_size, total_rows)}번째 데이터 ===")
    print(chunk[['text']])
    
    # 다음 20개를 보려면 엔터를 누르세요
    input("\n다음 20개를 보려면 Enter를 누르세요...")

In [20]:
high_ascii = df[cond1 &cond3]
len(high_ascii)

27

In [39]:
cond2 = df['uppercase_ratio'] < 0.8
high_ascii = df[cond1 & cond3]
len(high_ascii)

27

In [40]:
total_rows = len(high_ascii)
chunk_size = 20

for i in range(0, total_rows, chunk_size):
    chunk = high_ascii.iloc[i:i + chunk_size]
    print(f"\n=== {i+1}~{min(i+chunk_size, total_rows)}번째 데이터 ===")
    print(chunk[['text','uppercase_ratio' ]])
    
    # 다음 20개를 보려면 엔터를 누르세요
    input("\n다음 20개를 보려면 Enter를 누르세요...")


=== 1~20번째 데이터 ===
                                     text  uppercase_ratio
15         게시판 KISA 박민정 책임연구원 APTLD 이사 선출         1.000000
18                           개R전 연w정연H 작가         0.666667
68          MBC 류현진 특집 다큐 MLB 전설에게 듣는 류현진         1.000000
115                          최병륜 울산MBC 사장         1.000000
363              KAIST 온도따라 변하는 스마트 입자 개발         1.000000
582          UEFA 파리 생제르맹 FFP 위반 여부 조사 착수         1.000000
601                  美 매d g츠버그 강정호와 y별p T         0.200000
602        LA총영사관 UCLA·USC 한인학생회와 취업정보 공유         1.000000
638                        IoT로 쓰레기통 관리해요         0.666667
691       메시·호날두 UEFA 올해의 팀에 선정 EPL 선수 제로         1.000000
1132             KT카스 NBIoT 사업협력을 위한 업무협약         0.857143
1362        KT 멀티캠퍼스와 기가지니 SERICEO 서비스 론칭         1.000000
1505  日언론 고노 외상 美 USTR 대표와 회담 WTO 개혁 연대키로         1.000000
1692            KT·SKT 나란히 국제표준 생체인증 획득종합         1.000000
1707                        SKT 미래 고객 잡는다         1.000000
1788       中 RCEP 추진 박차 아태지역 통합에 TPP

In [ ]:
high_ascii = df[cond1]
high_ascii.head(10)